In [1]:
from ortools.sat.python import cp_model
import numpy as np
import math
import random

In [2]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, z):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__z = z
        self.__solution_count = 0


    def on_solution_callback(self):
        self.__solution_count += 1

        print(f'Solution {self.__solution_count}:', \
              ([i for i,s in self.__z.keys() if self.Value(self.__z[i,s]) == 1]))

    def solution_count(self):
        return self.__solution_count

In [6]:
# Function to create random grids of size n x n
def RandomGrid(n):
    '''
    n = Size of grid.
    '''
    # Random grid lights
    num = random.randint(1,n**2)
    lights = random.sample(range(0,n**2),num)
    print('Light(s) chosen: \n', sorted(lights))

    # Creating grid
    changes = [0,1,-1,n,-n]
    grid = np.zeros((1,n**2))[0]
    for i in sorted(lights):
        for j in changes:
            if i+j in range(n**2):
                if any([j == 1 and (i+j) % n == 0, j == -1 and i % n == 0]):
                    continue
                elif grid[i+j] == 0:
                    grid[i+j] = 1
                else:
                    grid[i+j] = 0          
    grid = grid.astype(int)
    print('Grid: \n',np.reshape(grid,(n,n)),'\n')
    return grid

In [11]:
#random.seed(43210)

def LightsOut(grid = None, rand_grid_size = None):

    '''
    Either input a grid or select a random grid to pass.
    Grid must be a square (rows = cols).
    Rand_grid: 
    '''

    if rand_grid_size != None:
        grid = RandomGrid(rand_grid_size)

    n = int(math.sqrt(len(grid)))

    #print(np.reshape(grid,(n,n)),'\n')

    # Creates the model and set solver
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()

    # Stages
    S = range(n**2)

    # Each cell in the grid
    x = {(i,s):model.NewBoolVar(f"x_{i}_{s}") for i in S for s in S}
    # Light that is switched
    z = {(i,s):model.NewBoolVar(f"z_{i}_{s}") for i in S for s in S}
    # Termination Condition (0 = continue, 1 = terminate)
    term = {(s):model.NewBoolVar(f"term_{s}") for s in S}

    changes = [0, 1, -1, n, -n]

    model.AddAtLeastOne([term[s] for s in S])

    for i in S:
        model.Add(x[i,0] == grid[i])
        model.AddAtMostOne([z[i,s] for s in S])
    
    for s in S:
        model.Add(sum(x[i,s] for i in S) == 0).OnlyEnforceIf(term[s])
        model.Add(sum(x[i,s] for i in S) != 0).OnlyEnforceIf(term[s].Not())
        if s>0:
            if s != S:
                model.Add(term[s-1] <= term[s])
            model.AddAtMostOne([z[i,s] for i in S])
            for i in S:
                lst = []
                for j in changes:
                    if i+j in S:
                        if any([j == 1 and (i+j) % n == 0, j == -1 and i % n == 0]):
                            continue
                        else:
                            model.Add(x[i+j,s-1].Not() == x[i+j,s]).OnlyEnforceIf(z[i,s])
                            lst.append((i+j))
                model.Add(x[i,s-1] == x[i,s]).OnlyEnforceIf([z[k,s].Not() for k in lst])

    # Incorporate objective to minimize number of lights chosen to solve problem
    # model.Minimize(sum(term[s].Not() for s in S))

    # Viewing the status of the solver
    solver.parameters.enumerate_all_solutions = False

    solution_printer = VarArraySolutionPrinter(z)

    status = solver.Solve(model, solution_printer)

    print(f"Status = {solver.StatusName(status)}")

    # Printing each stage of the solution
    # sol = [solver.value(x[i,s]) for s in S for i in S]
    # sol = np.reshape(sol,(n**2,n,n))
    # print(sol)


if __name__ == "__main__":

    ###############################
    ####### Default Grids ########
    ###############################
    # 7 x 7 grid
    # grid = [1,1,1,1,0,1,0,
    #         1,1,0,1,1,0,1,
    #         1,1,1,0,1,1,1,
    #         1,1,1,1,1,1,0,
    #         1,0,0,0,0,1,1,
    #         0,1,1,1,1,1,1,
    #         0,0,1,0,0,0,1]

    # 5 x 5 grid
    # grid = [1,1,1,1,1,
    #         1,0,1,1,0,
    #         0,0,0,0,1,
    #         0,1,1,1,0,
    #         0,0,0,1,0]
    
    # 3 x 3 grid
    # grid = [0,1,1,
    #         1,1,1,
    #         1,1,0]

    # Choose which method to create grid
    LightsOut(grid = None, rand_grid_size=5)

Light(s) chosen: 
 [0, 1, 3, 7, 11, 13, 15, 16, 17, 18, 19, 20, 21, 22, 24]
Grid: 
 [[0 0 1 1 1]
 [1 1 1 1 0]
 [0 0 0 0 0]
 [1 1 0 0 1]
 [1 0 1 1 0]] 

Solution 1: [4, 7, 10, 14, 15, 16, 17, 18, 19, 22, 23]
Status = OPTIMAL
